In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
import pandas as pd

In [ ]:
url_base = 'https://www.rabota.ru'
url ='https://www.rabota.ru/?query=Data%20Scientist&sort=relevance'

In [ ]:
request = requests.get(url)
if request.status_code == 200:
    print('Load is completed')
else:
    print('Load is not completed')

In [ ]:
page = BeautifulSoup(request.text, 'html.parser')

# Calculate the number of pages
pages = page.find_all('a', class_='pagination-list__item')
"""Number pages per vacancies"""
pages_lst = []
for page in pages:
    pages_lst.append(int(*page.text.split()))
    
page_min = min(pages_lst)
page_max = max(pages_lst)

print(f'max number of pages: {page_max}')

In [ ]:
urls = []
    
for page in range(1, page_max+1):
    urls.append(url + '&page=' + str(page))

urls

In [ ]:
final_titles = []
final_links = []

In [ ]:
request = requests.get(urls[10])
print(urls[1], request.status_code)
page = BeautifulSoup(request.text, 'html.parser')

In [ ]:
titles = page.find_all('h3', class_='vacancy-preview-card__title')
links = page.find_all('a', class_="vacancy-preview-card__title_border")

In [ ]:
titles_lst = [title.text.strip() for title in titles]
links_lst = [(url_base + link.get('href')) for link in links]

In [ ]:
def collect_vacancies(url, url_base):
    request = requests.get(url)
    if request.status_code == 200:
        print('Load is completed')
    else:
        print('Load is not completed')

    page = BeautifulSoup(request.text, 'html.parser')

    # Calculate the number of pages
    pages = page.find_all('a', class_='pagination-list__item')
    """Number pages per vacancies"""
    pages_lst = []
    for page in pages:
        pages_lst.append(int(*page.text.split()))
    
    page_min = min(pages_lst)
    page_max = max(pages_lst)

    print(f'max number of pages: {page_max}')

    urls = []
    
    for page in range(1, page_max+1):
        urls.append(url + '&page=' + str(page))

    final_titles = []
    final_links = []

    for u in urls:
        request = requests.get(u)
        page = BeautifulSoup(request.text, 'html.parser')
        titles = page.find_all('h3', class_='vacancy-preview-card__title')
        links = page.find_all('a', class_="vacancy-preview-card__title_border")
        titles_lst = [title.text.strip() for title in titles]
        links_lst = [(url_base + link.get('href')) for link in links]

    final_titles.extend(titles_lst)
    final_links.extend(links_lst)    
    

    # vacancy_dict = {x.text.strip():(url_base+y.get('href')) for x, y in zip(titles, links)}

    return final_titles, final_links

In [ ]:
titles, locations, salaries, dates = [], [], [], []

In [ ]:
for link in final_links:
    request = requests.get(link)
    page = BeautifulSoup(request.text, 'html.parser')

    try:
        title = page.find('h1', class_='vacancy-card__title').text.strip()
    except AttributeError:
        title = None
    try:
        location = page.find('span', class_='vacancy-requirements__city').text.strip()
    except AttributeError:
        location = None
    try:
        salary = page.find('h3', class_='vacancy-card__salary').text.strip()
    except AttributeError:
        salary = None
    try:
        date = page.find('span', class_='vacancy-system-info__updated-date').text.strip()
    except:
        date=None
    

    titles.append(title)
    locations.append(location)
    salaries.append(salary)
    dates.append(date)

In [ ]:
df = pd.DataFrame({'titles':titles, 'locations':locations, 'salaries':salaries, 'dates':dates})


In [ ]:
df.to_csv('rabota.csv', encoding='utf-8', index=False)